In [2]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import twitter
import json
import pandas as pd

In [3]:
import tweepy
import csv
import json

In [1]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python


## cloned repo:

https://github.com/Jefferson-Henrique/GetOldTweets-python

### local import

In [884]:
import sys
if sys.version_info[0] < 3:
    import got
else:
    import got3 as got

In [916]:
%cd /Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python


In [917]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python


In [918]:
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_token = info['ACCESS_KEY']
    access_token_secret = info['ACCESS_SECRET']


In [919]:
api = twitter.Api(consumer_key,consumer_secret,access_token,access_token_secret)

#It will print all your twitter profile data plus all your tweets!- uncomment below:

print('my twitter info & tweete:','\n',api.VerifyCredentials())

my twitter info & tweete: 
 {"created_at": "Sat Apr 08 04:23:41 +0000 2017", "default_profile": true, "default_profile_image": true, "favourites_count": 6, "followers_count": 5, "friends_count": 94, "id": 850564782310608896, "id_str": "850564782310608896", "name": "Natallia", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "natallia112202", "status": {"created_at": "Tue Jul 16 00:03:41 +0000 2019", "id": 1150918891905806336, "id_str": "1150918891905806336", "in_reply_to_screen_name": "realDonaldTrump", "in_reply_to_status_id": 1150879404593205249, "in_reply_to_user_id": 25073877, "lang": "en",

In [920]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret )
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)



In [921]:
api.get_user('Natallia112202').id

850564782310608896

### Function to extract users tweets

In [922]:
def get_user_tweets(name,num_tw):
        tweetCriteria = got.manager.TweetCriteria().setUsername(name).setMaxTweets(num_tw)
        all_tw = got.manager.TweetManager.getTweets(tweetCriteria)
        return [all_tw[i].text for i in range(len(all_tw))]


In [923]:
get_user_tweets('Natallia112202',2) 

['Wow, they are really outraged. The provocation surely worked. They got trapped like little bunnies.',
 'poor cry babies, red revolutionaries, they cater to their constituency - tribal and uneducated folks. Who else would buy the rhetoric of emotional accusations, total disregard of facts and class envy? Trump was not even the first to state that "if you do not like it - go back"']

In [897]:

#####
get_user_tweets('barackobama',5) 
#######
#create a list of user_ids
u_names = ['barackobama','Natallia112202']
user_id = []
i = 0

for item in u_names:
    try:
        user_id.append(api.get_user(screen_name = item).id)
    except:
        i = i + 1
        print(item)
        print("badvalue")
        user_id.append("NA")
print('Users IDs',user_id)

######
user_follow = []
i = 0

for item in u_names:
    try:
        user_follow.append(api.get_user(screen_name = item).followers_count)
    except:
        i = i + 1
        print(item)
        print("badvalue")
        user_id.append("NA")  
print("Users num of followers",user_follow)

Users IDs [813286, 850564782310608896]
Users num of followers [107110149, 5]


### Estracting users screen_names from csv files

In [924]:
p_types = 'ISTJ,ISFJ,INFJ,INTJ,ISTP,ISFP,INFP,INTP,ESTP,ESFP,ENFP,ENTP,ESTJ,ESFJ,ENFJ,ENTJ'
## transformed into python list
p_list = p_types.split(',', 16)
#len(p_list)
p_list

['ISTJ',
 'ISFJ',
 'INFJ',
 'INTJ',
 'ISTP',
 'ISFP',
 'INFP',
 'INTP',
 'ESTP',
 'ESFP',
 'ENFP',
 'ENTP',
 'ESTJ',
 'ESFJ',
 'ENFJ',
 'ENTJ']

##### Clone folder:
https://github.com/Jefferson-Henrique/GetOldTweets-python
-  in your terminal window for each of the MBTI types(see above)type this:


 python Exporter.py --querysearch "I am an *" --since 2006-05-01 --until 2019-05-01 --maxtweets 1000  --output *.csv

    where * is one of MBTI types


In [925]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python


In [926]:
def parse_csv(CSV):
    full_tw = []
    with open(CSV) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            full_tw.append(row)
    full_tw_noH = full_tw[1:]
    tot_rows = len(full_tw_noH)
    users = []
    for i in range(len(full_tw_noH)):
        string_row = " ".join(str(x) for x in full_tw_noH[i])
        user = string_row.split(';')[-1].split('//')[1].split('/')[1]
        users.append(user)
    return tot_rows,users
        
    

### can parse one csv file to extract user names ...

In [927]:
print('total number of names',parse_csv("ENTJ.csv")[0])

total number of names 1000


In [928]:
print('first name on the name list',parse_csv("ENTJ.csv")[1][0])

first name on the name list Roland00Address


### ... But cannot deal with multiple files as this:

In [744]:
p_list[8:12]

['ESTP', 'ESFP', 'ENFP', 'ENTP']

In [ ]:
## even if htere are files in dir

In [929]:
!ls -l

total 406000
-rw-r--r--   1 Natallia  staff     982377 Jul 15 17:50 DF_NS.ipynb
-rw-r--r--   1 Natallia  staff     983398 Jul 13 19:38 DF_NS_GCP.ipynb
-rw-r--r--   1 Natallia  staff   23291620 Jul 15 16:41 DF_NS_fill_data.ipynb
-rw-r--r--   1 Natallia  staff      46818 Jul 15 13:23 DF_NS_fill_data_2_GCP.ipynb
-rw-r--r--   1 Natallia  staff      49270 Jul 15 13:22 DF_NS_fill_data_GCP.ipynb
-rw-r--r--   1 Natallia  staff     237661 Jul 12 00:59 ENTJ.csv
-rw-r--r--   1 Natallia  staff      79496 Jul 12 00:49 ESTP.csv
-rw-r--r--@  1 Natallia  staff       2038 Jul 10 19:56 Exporter.py
-rw-r--r--@  1 Natallia  staff     325699 Jul 11 21:53 FAAPT_01239_01289AB-final.ipynb
-rw-r--r--   1 Natallia  staff       1085 Jul 10 19:26 LICENSE
-rw-r--r--@  1 Natallia  staff       1320 Jul 10 19:26 Main.py
-rw-r--r--   1 Natallia  staff       4072 Jul 10 19:26 README.md
-rw-r--r--@  1 Natallia  staff      15566 Jul 14 18:12 Search_07122019_terminal_0utput.docx
drwxr-xr-x  18 Natallia  staff        576 J

In [735]:
for it in p_list:
    parse_csv(it)[1]
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'ISTJ'

In [930]:
pathName = os.getcwd()
pathName

'/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python'

In [931]:
!ls -l

total 406000
-rw-r--r--   1 Natallia  staff     982377 Jul 15 17:50 DF_NS.ipynb
-rw-r--r--   1 Natallia  staff     983398 Jul 13 19:38 DF_NS_GCP.ipynb
-rw-r--r--   1 Natallia  staff   23291620 Jul 15 16:41 DF_NS_fill_data.ipynb
-rw-r--r--   1 Natallia  staff      46818 Jul 15 13:23 DF_NS_fill_data_2_GCP.ipynb
-rw-r--r--   1 Natallia  staff      49270 Jul 15 13:22 DF_NS_fill_data_GCP.ipynb
-rw-r--r--   1 Natallia  staff     237661 Jul 12 00:59 ENTJ.csv
-rw-r--r--   1 Natallia  staff      79496 Jul 12 00:49 ESTP.csv
-rw-r--r--@  1 Natallia  staff       2038 Jul 10 19:56 Exporter.py
-rw-r--r--@  1 Natallia  staff     325699 Jul 11 21:53 FAAPT_01239_01289AB-final.ipynb
-rw-r--r--   1 Natallia  staff       1085 Jul 10 19:26 LICENSE
-rw-r--r--@  1 Natallia  staff       1320 Jul 10 19:26 Main.py
-rw-r--r--   1 Natallia  staff       4072 Jul 10 19:26 README.md
-rw-r--r--@  1 Natallia  staff      15566 Jul 14 18:12 Search_07122019_terminal_0utput.docx
drwxr-xr-x  18 Natallia  staff        576 J

In [694]:
fileNames = os.listdir(pathName)
fileNames

['twitter_credentials.json',
 '.DS_Store',
 'stability_faapt_feb2019.ipynb',
 'LICENSE',
 'requirements.txt',
 'Exporter.py',
 'got',
 'DF_NS.ipynb',
 'old_data',
 'README.md',
 'ENTJ.csv',
 '.gitignore',
 'FAAPT_01239_01289AB-final.ipynb',
 'got3',
 '.ipynb_checkpoints',
 'exporter_help_text.txt',
 '.git',
 'Main.py',
 'data']

In [932]:
numFiles = []
for fName in fileNames:
    if fName.endswith(".csv"):
        numFiles.append(fName)
numFiles

['ENFP.csv',
 'INFP.csv',
 'ISFP.csv',
 'ESFP.csv',
 'ISTP.csv',
 'ESTP.csv',
 'ENTP.csv',
 'INTP.csv',
 'INTJ.csv',
 'ENTJ.csv',
 'ESTJ.csv',
 'ISTJ.csv',
 'ESFJ.csv',
 'ISFJ.csv',
 'INFJ.csv',
 'ENFJ.csv']

In [701]:
for i in numFiles:
    file = open(os.path.join(pathName, i))
    reader = csv.reader(file, delimiter=',')
    for item in reader:
        print(item)

['username;date;retweets;favorites;text;geo;mentions;hashtags;id;permalink']
[';2019-04-30 13:31;0;1;"Only selectively useful some of the time. It is either hyper-useful or not useful at all and it can get in the way of other things. Very polar. I am reminded of this. Some times they want to be an ENTJ ', ' sometimes an INFJ', ' and other times an ESTP. https:// twitter.com/amor_vincit_1/ status/1122326654481600512 …";;;;"1123323997272399872";https://twitter.com/Roland00Address/status/1123323997272399872']
[';2019-04-25 21:45;0;0;"I am having a sweet and dirty dream that a perfectly lovely gentle ginger ENTJ girl', ' throwing aside an aquamarine chemise', ' sings softly to an attentive but magical calico mustang.";;;;"1121636324111306752";https://twitter.com/MBTIhorsewomen/status/1121636324111306752']
[';2019-04-25 17:47;0;7;"I', ' an humble ENTJ ', ' today learned I am not good at fEeLiNgS";;;;"1121576457770131457";https://twitter.com/ashytin_/status/1121576457770131457']
[';2019-04-2

In [717]:
pathName = os.getcwd()

myFiles = []
fileNames = os.listdir(pathName)
for fName in fileNames:
    if fName.endswith(".csv"):
        myFiles.append(fName)

for i in myFiles:
    file = open(os.path.join(pathName, i))
    reader = csv.reader(file, delimiter=',')
    for item in reader:
        print(item)

['username;date;retweets;favorites;text;geo;mentions;hashtags;id;permalink']
[';2019-04-30 13:31;0;1;"Only selectively useful some of the time. It is either hyper-useful or not useful at all and it can get in the way of other things. Very polar. I am reminded of this. Some times they want to be an ENTJ', ' sometimes an INFJ', ' and other times an ESTP . https:// twitter.com/amor_vincit_1/ status/1122326654481600512 …";;;;"1123323997272399872";https://twitter.com/Roland00Address/status/1123323997272399872']
[';2019-04-12 19:14;0;0;"I am having a sweet and lovely dream that a fun chocoholic ESTP girl', ' carefully unbuttoning an indigo white kimono', ' gently patting the nose of a scientifically brilliant chestnut mustang.";;;;"1116887430165037056";https://twitter.com/MBTIhorsewomen/status/1116887430165037056']
[';2019-04-12 16:29;0;1;"To that I would say: I am an ESTP the Myers-Briggs world. These two paragraphs from http:// 16personalities.com is a great summary as it applies here...."

In [708]:
pathName = os.getcwd()## get working directory
myFiles = []
fileNames = os.listdir(pathName)# get list of available in ths dir files
## choose only csv files and fill in myFiles list)
for fName in fileNames:
    if fName.endswith(".csv"):
        myFiles.append(fName)

In [906]:
myFiles

['ENFP.csv',
 'INFP.csv',
 'ISFP.csv',
 'ESFP.csv',
 'ISTP.csv',
 'ESTP.csv',
 'ENTP.csv',
 'INTP.csv',
 'INTJ.csv',
 'ENTJ.csv',
 'ESTJ.csv',
 'ISTJ.csv',
 'ESFJ.csv',
 'ISFJ.csv',
 'INFJ.csv',
 'ENFJ.csv']

In [912]:
pathName = os.getcwd()## get working directory

print('working dir:',pathName)

working dir: /Users/Natallia/Library/Mobile Documents/com~apple~CloudDocs/.Trash/data 3.30.38 PM


### move all csv files from directory above to "data" directory

### Change wd to whatever your store your csv files

In [933]:
my_dir = 'data'
os.chdir(my_dir)

In [934]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python/data


## Parse all csv files in the 'data' directory and extract user names

In [935]:
pathName = os.getcwd()## get current working directory
## crete empty list of files I want to work with
myFiles = []
fileNames = os.listdir(pathName)# get list of available in ths dir files
## choose only csv files and fill in myFiles list)
for fName in fileNames:
    if fName.endswith(".csv"):
        myFiles.append(fName)
print('My files list',myFiles )
## create list to collect lines of csv file
full_tw = []
num_rows_list = []
##iterate through files in "myFiles" - should be 16 csv files!
for item in myFiles:
    ## join path name with each file name in myFiles
    file = open(os.path.join(pathName, item))
    ## read csv file one by one
    reader = csv.reader(file, delimiter=',')
    ## remove header line from each file/item
    ## count rows in each csv minus header
    row_num_csv = 0
    for row in reader:
       
        if row[0] != 'username;date;retweets;favorites;text;geo;mentions;hashtags;id;permalink':
            
            full_tw.append(row)
        row_num_csv += 1
    num_rows_list.append(row_num_csv-1)
tot_rows = len(full_tw)
    ## create empty users' screen_name list 
users = []
    ### start collecting user's screen names from all csv files in hte directory
    
for i in range(tot_rows):
    string_row = " ".join(str(x) for x in full_tw[i])
    user = string_row.split(';')[-1].split('//')[1].split('/')[1]
    users.append(user)

    
    
    

My files list ['ENFP.csv', 'INFP.csv', 'ISFP.csv', 'ESFP.csv', 'ISTP.csv', 'ESTP.csv', 'ENTP.csv', 'INTP.csv', 'INTJ.csv', 'ENTJ.csv', 'ESTJ.csv', 'ISTJ.csv', 'ESFJ.csv', 'ISFJ.csv', 'INFJ.csv', 'ENFJ.csv']


## Number of rows (minus header) ineach csv file:

In [936]:
num_rows_list

[1000,
 1000,
 738,
 631,
 556,
 348,
 1000,
 1000,
 1000,
 1000,
 583,
 1000,
 689,
 981,
 1000,
 1000]

In [939]:
MBTI_users_by_type = pd.DataFrame({'files to process':myFiles,'rows in file': num_rows_list})

MBTI_users_by_type

,files to process,rows in file
0,ENFP.csv,1000
1,INFP.csv,1000
2,ISFP.csv,738
3,ESFP.csv,631
4,ISTP.csv,556
5,ESTP.csv,348
6,ENTP.csv,1000
7,INTP.csv,1000
8,INTJ.csv,1000
9,ENTJ.csv,1000


In [941]:
proc_order = MBTI_users_by_type.to_csv('../file_processing_order_MBTI.csv')


In [747]:
len(users)

13526

In [748]:
full_tw[0]

[';2019-04-30 09:56;0;1;"I have no shame and little guile when it comes to meeting people live',
 ' but I am an ENFP so that is par for the course for us.";;;;"1123269839026642944";https://twitter.com/BillTooke/status/1123269839026642944']

In [749]:
%ls -l

total 9648
-rw-r--r--  1 Natallia  staff  1135934 Jul 12 13:50 16_CSVFiles.zip
-rw-r--r--  1 Natallia  staff   224876 Jul 12 00:58 ENFJ.csv
-rw-r--r--  1 Natallia  staff   250316 Jul 12 00:52 ENFP.csv
-rw-r--r--  1 Natallia  staff   237661 Jul 12 00:59 ENTJ.csv
-rw-r--r--  1 Natallia  staff   235724 Jul 12 00:54 ENTP.csv
-rw-r--r--  1 Natallia  staff   154281 Jul 12 00:56 ESFJ.csv
-rw-r--r--  1 Natallia  staff   139708 Jul 12 00:51 ESFP.csv
-rw-r--r--  1 Natallia  staff   134474 Jul 12 00:55 ESTJ.csv
-rw-r--r--  1 Natallia  staff    79496 Jul 12 00:49 ESTP.csv
-rw-r--r--  1 Natallia  staff   275880 Jul 12 00:39 INFJ.csv
-rw-r--r--  1 Natallia  staff   258147 Jul 12 00:47 INFP.csv
-rw-r--r--  1 Natallia  staff   275007 Jul 12 00:41 INTJ.csv
-rw-r--r--  1 Natallia  staff   241992 Jul 12 00:48 INTP.csv
-rw-r--r--  1 Natallia  staff   218016 Jul 12 00:35 ISFJ.csv
-rw-r--r--  1 Natallia  staff   162749 Jul 12 00:45 ISFP.csv
-rw-r--r--@ 1 Natallia  staff   226355 Jul 12 00:29 ISTJ.csv
-rw-r-

In [942]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python/data


### Build DF

In [943]:
##current dir (make sure it is dir where your dta is - scv files
## - we will need their names)
## set current dir
my_dir = '.'
filelist_names = []
filesList = []
os.chdir(my_dir)

# Step 2: Build up list of files:
for files in glob.glob("*.csv"):
    fileName, fileExtension = os.path.splitext(files)
    filelist_names.append(fileName) #filename without extension
    filesList.append(files) #filename with extension


In [526]:
!pwd

/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python/data


In [944]:
filelist_names

['ENFP',
 'INFP',
 'ISFP',
 'ESFP',
 'ISTP',
 'ESTP',
 'ENTP',
 'INTP',
 'INTJ',
 'ENTJ',
 'ESTJ',
 'ISTJ',
 'ESFJ',
 'ISFJ',
 'INFJ',
 'ENFJ']

In [945]:
filesList

['ENFP.csv',
 'INFP.csv',
 'ISFP.csv',
 'ESFP.csv',
 'ISTP.csv',
 'ESTP.csv',
 'ENTP.csv',
 'INTP.csv',
 'INTJ.csv',
 'ENTJ.csv',
 'ESTJ.csv',
 'ISTJ.csv',
 'ESFJ.csv',
 'ISFJ.csv',
 'INFJ.csv',
 'ENFJ.csv']

In [759]:
filesList[0]

'ENFP.csv'

#### Construct the list of all dataframes to fill in with data

##### ways to create empty DF

https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it

In [986]:
num_rows_list

[1000,
 1000,
 738,
 631,
 556,
 348,
 1000,
 1000,
 1000,
 1000,
 583,
 1000,
 689,
 981,
 1000,
 1000]

In [967]:
d = {}
for name,num_rows in zip(filelist_names,num_rows_list):
    d[name] = pd.DataFrame(columns = ['MBTI','created_prof','user_screen_name','tweets','id_','loc',\
                            'descr','prot','followers','follow','friends','fav',\
                             'timeZ','geo_en','prof_bg_col','prof_bkim_url',\
                          'prof_image_url'],index=range(num_rows))
    #d[name].MBTI = list([name]*num_rows)
    
    

In [1034]:
#d['ISFP'].MBTI

In [969]:
d

{'ENFP':     MBTI created_prof user_screen_name tweets  id_  loc descr prot followers  \
 0    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 1    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 2    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 3    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 4    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 5    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 6    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 7    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 8    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 9    NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 10   NaN          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 11   NaN          N

### one of DF in the list (d) of dataframes:

In [970]:
d['ISTJ']

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [971]:
d['ENFP'].shape

(1000, 17)

In [972]:
d['ENFP'].iloc[0,15]

nan

In [983]:
d['ENFP'].MBTI[1:8]

1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
Name: MBTI, dtype: object

### file processing order and number of rows in each file(df) per MBTI type

In [987]:
filelist_names

['ENFP',
 'INFP',
 'ISFP',
 'ESFP',
 'ISTP',
 'ESTP',
 'ENTP',
 'INTP',
 'INTJ',
 'ENTJ',
 'ESTJ',
 'ISTJ',
 'ESFJ',
 'ISFJ',
 'INFJ',
 'ENFJ']

In [1007]:
filelist_names

['ENFP',
 'INFP',
 'ISFP',
 'ESFP',
 'ISTP',
 'ESTP',
 'ENTP',
 'INTP',
 'INTJ',
 'ENTJ',
 'ESTJ',
 'ISTJ',
 'ESFJ',
 'ISFJ',
 'INFJ',
 'ENFJ']

In [950]:


print('total number of users',sum(num_rows_list))

total number of users 13526


In [989]:
for t in filelist_names:
    d[t].MBTI = t

In [990]:
d

{'ENFP':      MBTI created_prof user_screen_name tweets  id_  loc descr prot followers  \
 0    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 1    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 2    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 3    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 4    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 5    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 6    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 7    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 8    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 9    ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 10   ENFP          NaN              NaN    NaN  NaN  NaN   NaN  NaN       NaN   
 11   EN

### Concatenate all the dataframes in d (list of df)

In [1010]:
for item in filelist_names:
    dff = 'd'+'['+'"'+item+'"'+']'
    print(dff)

d["ENFP"]
d["INFP"]
d["ISFP"]
d["ESFP"]
d["ISTP"]
d["ESTP"]
d["ENTP"]
d["INTP"]
d["INTJ"]
d["ENTJ"]
d["ESTJ"]
d["ISTJ"]
d["ESFJ"]
d["ISFJ"]
d["INFJ"]
d["ENFJ"]


In [1015]:
type(d["ENFP"])

pandas.core.frame.DataFrame

In [1019]:
df_NS = pd.concat([d["ENFP"],d["INFP"],d["ISFP"],d["ESFP"],d["ISTP"],d["ESTP"],d["ENTP"],d["INTP"],d["INTJ"],d["ENTJ"],d["ESTJ"],d["ISTJ"],d["ESFJ"],d["ISFJ"],d["INFJ"],d["ENFJ"]]).reset_index(drop=True)
#df_NS.MBTI[1999:2009]

In [1024]:
file_proc = pd.read_csv('../file_processing_order_MBTI.csv',index_col = 0)
#print(file_proc)
starting_p = 0
list_sp = []
for item,mbti in zip(list(file_proc['rows in file']),list(file_proc['files to process'])):
    starting_p +=item 
    list_sp.append(starting_p-1)
    print('last row in file ',mbti,'is row # ',starting_p-1, 'in full dataset')
    

last row in file  ENFP.csv is row #  999 in full dataset
last row in file  INFP.csv is row #  1999 in full dataset
last row in file  ISFP.csv is row #  2737 in full dataset
last row in file  ESFP.csv is row #  3368 in full dataset
last row in file  ISTP.csv is row #  3924 in full dataset
last row in file  ESTP.csv is row #  4272 in full dataset
last row in file  ENTP.csv is row #  5272 in full dataset
last row in file  INTP.csv is row #  6272 in full dataset
last row in file  INTJ.csv is row #  7272 in full dataset
last row in file  ENTJ.csv is row #  8272 in full dataset
last row in file  ESTJ.csv is row #  8855 in full dataset
last row in file  ISTJ.csv is row #  9855 in full dataset
last row in file  ESFJ.csv is row #  10544 in full dataset
last row in file  ISFJ.csv is row #  11525 in full dataset
last row in file  INFJ.csv is row #  12525 in full dataset
last row in file  ENFJ.csv is row #  13525 in full dataset


In [1025]:
print('points of MBTI label changes',list_sp)
    

points of MBTI label changes [999, 1999, 2737, 3368, 3924, 4272, 5272, 6272, 7272, 8272, 8855, 9855, 10544, 11525, 12525, 13525]


In [1026]:
df_NS[1999:2001]

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
1999,INFP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,ISFP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add user screen_names

In [1027]:

df_NS['user_screen_name'] = users

In [1028]:
df_NS.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFP,NaN,BillTooke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENFP,NaN,dougie0216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENFP,NaN,kimbetech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENFP,NaN,honeyBklein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENFP,NaN,tanishatray4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1029]:
df_NS.shape

(13526, 17)

In [1031]:
df_NS.infer_objects().dtypes

MBTI                 object
created_prof        float64
user_screen_name     object
tweets              float64
id_                 float64
loc                 float64
descr               float64
prot                float64
followers           float64
follow              float64
friends             float64
fav                 float64
timeZ               float64
geo_en              float64
prof_bg_col         float64
prof_bkim_url       float64
prof_image_url      float64
dtype: object

In [1035]:
personalities_df = df_NS.to_csv ('../df_users_NS.csv', index = None, header=True)


In [1032]:
!pwd


/Users/Natallia/Documents/R/W210_capstone/Project/GetOldTweets-python/data


### Using function get_user_tweets(name,num_tw) to add users 200 tweets

### Cannot collect all at once

In [ ]:
## since I know there are errors in collection starting with user[7429],
## split users into U1 = users[:7400] and U2 = users[7401:]
## colect twe_u1 = twe = [get_user_tweets(name,200) for name in users]
## colelct U2 as see below:

In [ ]:
tweets = []
i = 0
for item in tqdm(U2):
    try:
        
        tweets.append(get_user_tweets(item,200))
        
    except:
        i += 1
        
        print("badvalue for item",item)
        tweets.append("NA") 
        
        

In [838]:
twe = [get_user_tweets(name,200) for name in users]

Twitter weird response. Try to see on browser: https://twitter.com/search?q=%20from%3AIlliniBizDean&src=typd
Unexpected error: <class 'urllib.error.HTTPError'>


SystemExit: 

//anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [1033]:
df_NS[df_NS['user_screen_name']=='IlliniBizDean']

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
7429,ENTJ,NaN,IlliniBizDean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [866]:
import time
from tqdm import tqdm # this is used to track progress
from tqdm.autonotebook import tqdm
tqdm.pandas()

//anaconda/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



In [878]:
users[7429]

'IlliniBizDean'

In [879]:
df_NS1 = df_NS.drop([7429]).head()

In [880]:
df_NS1[df_NS1['user_screen_name']=='IlliniBizDean']

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url


In [881]:
df_NS1.head()

,MBTI,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,ENFJ,NaN,BillTooke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENFJ,NaN,dougie0216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENFJ,NaN,kimbetech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENFJ,NaN,honeyBklein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENFJ,NaN,tanishatray4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save file

In [882]:
personalities_df = df_NS1.to_csv ('../personalities_df_NS.csv', index = None, header=True)


In [853]:
users1 = users[:10]


In [854]:
users1

['BillTooke',
 'dougie0216',
 'kimbetech',
 'honeyBklein',
 'tanishatray4',
 '100YrLifeStyle',
 'Ally_759_',
 'craigymcmillan',
 'kamidenks',
 'luikaOJO']

In [846]:
len(set(users))

10339

In [867]:
users_20 = users[:20]

In [868]:

id_20 = [api.get_user(screen_name = item).id for item in tqdm(users_20)]


In [ ]:
d1['id_'] = [api.get_user(screen_name = item).id for item in tqdm(users)]



https://stackoverflow.com/questions/41624241/pandas-split-dataframe-into-two-dataframes

In [586]:
#parsed[0]

## Mini set

2017-04-08 04:23:41


In [625]:
print(api.get_user('Natallia112202').created_at )



2017-04-08 04:23:41


In [659]:
## created empty dataframe
d1 = pd.DataFrame(columns = ['user','created_prof','user_screen_name','tweets','id_','loc',\
                            'descr','prot','followers','follow','friends','fav',\
                             'timeZ','geo_en','prof_bg_col','prof_bkim_url',\
                            'prof_image_url'])
d1

,user,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url


### attributes to get with api.get_user()
- 'tweets'(limited by timeframe)
- 'user'
- 'user_screen_name'
- 'created_prof'
- 'id'
- 'loc'
- 'descr'
- 'followers'
- 'follow'


### ADD User data

## All info about user by API
https://stackoverflow.com/questions/15628535/how-can-i-retrieve-all-tweets-and-attributes-for-a-given-user-using-python

In [660]:
usr = ['Obama','Natallia']
scr = ['barackobama','Natallia112202']

In [661]:
for name,sc_name in zip(usr,scr):
    d1['user'] = usr
    d1['user_screen_name'] = scr
    
print('User column length:',len(d1['user'].tolist()))
print('User_screen_name length:',len(d1['user_screen_name'].tolist()))
    
    

User column length: 2
User_screen_name length: 2


In [662]:
d1

,user,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,Obama,NaN,barackobama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Natallia,NaN,Natallia112202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Using function get_user_tweets(name,num_tw)

In [883]:
### FIll intweet column:
tw = [get_user_tweets(name,4) for name in scr]
d1['tweets']=tw
d1

,user,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,Obama,NaN,barackobama,[The @ObamaFoundation is helping a new generat...,813286,"Washington, DC","Dad, husband, President, citizen.",False,107087372,False,613320,11,None,False,77B0DC,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8225477323...
1,Natallia,NaN,Natallia112202,[Wow. Leave ICE employees alone... They are cl...,850564782310608896,,,False,5,False,94,6,None,False,F5F8FA,None,http://abs.twimg.com/sticky/default_profile_im...


In [666]:
### all 4 obama Tweets
d1.iloc[0,3]

['The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn',
 'Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE',
 'Yes! Fourth star. Back to back. Congrats to the record breakers on the @USWNT , an incredible team that’s always pushing themselves—and the rest of us—to be even better. Love this team. #OneNationOneTeam',
 'Happy Fourth of July, everybody! This is always a great day in the Obama family: a chance to celebrate America—and Malia’s birthday, too. Hope all of you are able to get some time with friends, family, and fireworks. pic.twitter.com/Gn9kVCCnuf']

### Add other attributes for a user using the user_screen names list (scr)

In [656]:
scr

['barackobama', 'Natallia112202']

In [669]:
d1['id_'] = [api.get_user(screen_name = item).id for item in scr]
d1['loc'] = [api.get_user(screen_name = item).location for item in scr]
d1['descr'] = [api.get_user(screen_name = item).description for item in scr]
d1['prot'] = [api.get_user(screen_name = item).protected for item in scr]
d1['followers'] = [api.get_user(screen_name = item).followers_count for item in scr]
d1['follow'] = [api.get_user(screen_name = item).following for item in scr]

d1['friends'] = [api.get_user(screen_name = item).friends_count for item in scr]             
    
d1['fav'] = [api.get_user(screen_name = item).favourites_count for item in scr]   
d1['timeZ'] = [api.get_user(screen_name = item).time_zone for item in scr] 
d1['geo_en'] = [api.get_user(screen_name = item).geo_enabled for item in scr] 
d1['prof_bg_col'] = [api.get_user(screen_name = item).profile_background_color for item in scr] 
d1['prof_bkim_url'] = [api.get_user(screen_name = item).profile_background_image_url for item in scr] 
d1['prof_image_url'] = [api.get_user(screen_name = item).profile_image_url for item in scr]


#### DF

In [670]:
d1

,user,created_prof,user_screen_name,tweets,id_,loc,descr,prot,followers,follow,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url
0,Obama,NaN,barackobama,[The @ObamaFoundation is helping a new generat...,813286,"Washington, DC","Dad, husband, President, citizen.",False,107087372,False,613320,11,None,False,77B0DC,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8225477323...
1,Natallia,NaN,Natallia112202,[Wow. Leave ICE employees alone... They are cl...,850564782310608896,,,False,5,False,94,6,None,False,F5F8FA,None,http://abs.twimg.com/sticky/default_profile_im...


In [671]:
images = d1['prof_image_url'].tolist()

In [672]:
#### Fix images (bring all to original type)
for i in range(len(images)):
    images[i] = images[i].replace('_normal', '')

In [673]:
d1['prof_image_url']=images

In [677]:
d1.iloc[1,16]

'http://abs.twimg.com/sticky/default_profile_images/default_profile.png'

### INFO TO GET WITH api.user_timeline

### Important attributes about tweets

##### INFO about tweets (cannot really use it bc we will have 200 tw/user - cannot be separate column)
 - 'Geo'
 - 'Source'(ipad? iphone?)
 - 'Favorited'
 - 'retweet_count

In [641]:
user = 'barackobama'
## if do not put count = it will get tweets from month ago to today

In [642]:
timeline = api.user_timeline(screen_name=user, include_rts=False, count=5)
for tweet in timeline:
    print ("ID:", tweet.id)
    print ("User ID:", tweet.user.id)
    print ("Text:", tweet.text)
    print ("Created:", tweet.created_at)
    print ("Geo:", tweet.geo)
    print ("Contributors:", tweet.contributors)
    print ("Coordinates:", tweet.coordinates) 
    print ("Favorited:", tweet.favorited)
    print ("In reply to screen name:", tweet.in_reply_to_screen_name)
    print ("In reply to status ID:", tweet.in_reply_to_status_id)
    print ("In reply to status ID str:", tweet.in_reply_to_status_id_str)
    print ("In reply to user ID:", tweet.in_reply_to_user_id)
    print ("In reply to user ID str:", tweet.in_reply_to_user_id_str)
    print ("Place:", tweet.place)
    print ("Retweeted:", tweet.retweeted)
    print ("Retweet count:", tweet.retweet_count)
    print ("Source:", tweet.source)
    print ("Truncated:", tweet.truncated)

ID: 1149008015716233216
User ID: 813286
Text: The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gat… https://t.co/azU3MXDYlN
Created: 2019-07-10 17:30:33
Geo: None
Contributors: None
Coordinates: None
Favorited: False
In reply to screen name: None
In reply to status ID: None
In reply to status ID str: None
In reply to user ID: None
In reply to user ID str: None
Place: None
Retweeted: False
Retweet count: 7071
Source: Twitter Media Studio
Truncated: True
ID: 1148940273411661824
User ID: 813286
Text: Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girl… https://t.co/L8OY8yP0Mi
Created: 2019-07-10 13:01:22
Geo: None
Contributors: None
Coordinates: None
Favorited: False
In reply to screen name: None
In reply to status ID: None
In reply to status ID str: None
In reply to user ID: None
In reply to user ID str: None
Place: None
Retweeted: False
Retweet count: 58842
Sour

In [581]:
##our  d - dictionary is the list of dataframes, currently empty
##our filelist_names is the list of names oth those dataframes (also, keys in d
## filesList - is the list of csv files generated previously
for df,name,i in zip(d,filelist_names,list(range(16))):
    df = d[name]
    
    u_names = parsed[i]
    #df['users']=u_names
    df['tweets'] = [get_user_tweets(name,4) for name in u_names]
    df['id_'] = [api.get_user(screen_name = item).id for item in u_names]
    df['loc'] = [api.get_user(screen_name = item).location for item in u_names]
    df['descr'] = [api.get_user(screen_name = item).description for item in u_names]
    df['prot'] = [api.get_user(screen_name = item).protected for item in u_names]
    df['followers'] = [api.get_user(screen_name = item).followers_count for item in u_names]
    df['friends'] = [api.get_user(screen_name = item).friends_count for item in u_names]             
    df['fav'] = [api.get_user(screen_name = item).favourites_count for item in u_names]   
    df['timeZ'] = [api.get_user(screen_name = item).time_zone for item in u_names] 
    df['geo_en'] = [api.get_user(screen_name = item).geo_enabled for item in u_names] 
    df['prof_bg_col'] = [api.get_user(screen_name = item).profile_background_color for item in u_names] 

    df['prof_bkim_url'] = [api.get_user(screen_name = item).profile_background_image_url for item in u_names] 
    df['prof_image_url'] = [api.get_user(screen_name = item).profile_image_url for item in u_names]
    df['follow'] = [api.get_user(screen_name = item).following for item in u_names]


    
    

ValueError: Length of values does not match length of index

In [582]:
df1 = d['INTJ']
df1.head()

,users,tweets,id_,loc,descr,prot,followers,friends,fav,timeZ,geo_en,prof_bg_col,prof_bkim_url,prof_image_url,follow


In [291]:
df = d['output_got_INTJ_200']
type(df)

pandas.core.frame.DataFrame

In [292]:
col = parse_csv("output_got_INTJ_200.csv")[1]
col

['thazin_98',
 'kjellgal',
 'modelerr',
 'TIL_ayg',
 'TIL_ayg',
 'red_ward13',
 'TIL_ayg',
 'awonderingpoet',
 'TennileCooper',
 'idkemam',
 'fluffyparkji',
 'LydiaMacfarlan',
 'artemidite',
 'killyalison',
 'Brixest',
 'KiyBee',
 'Roland00Address',
 'mzoiy',
 'innpidinnp',
 'Kagendo__',
 'akiwatarikainy',
 'Kawtheraka',
 'kanbanwaru',
 'AudreySimmons06',
 'Jeanthejust',
 'quemirasnojoda',
 'dayne_smith1',
 'denialt23',
 'Michell58951656',
 'fyuwudor',
 'SamaMeji',
 'MagpieDiviner',
 'entish2',
 'imcatnoone',
 'Michell58951656',
 'terdle19541',
 'timbayIiss',
 'yoemilyrey',
 'West221B',
 'alyssakeiko',
 'bea_minus',
 'jbrodley',
 'JulieSwift000',
 'aprilminty_',
 'Smlygrl87',
 'Roland00Address',
 'stufr67',
 'blissblar',
 'jdepler2',
 'bulldawgwarrior',
 'artemidite',
 'father_loofa',
 'KriticalCory',
 '_AlexPerdue',
 'Steviekouts12',
 'xbxba84',
 'dirkdisco',
 'AndrewYang',
 'AuthKhaliLaBre',
 'bornto_hula',
 'absolute_INTJ',
 'PetaCulverhouse',
 'lschmeiser',
 'Roland00Address',
 'Me

In [293]:

df['users']=parse_csv("output_got_INTJ_200.csv")[1]

In [308]:
#df

In [254]:
filesList

['output_got_MBTI.csv',
 'output_got_INTJ_200.csv',
 'output_got_ISTJ.csv',
 'since2006__ISTJ.csv']

In [256]:
parse_csv('output_got_MBTI.csv')[1]

['capitubsides']

In [250]:
for DF,f in zip(d,filesList):
    d[DF]=parse_csv(f)[1]

In [253]:
d['output_got_MBTI']

['capitubsides']

#### remove header line

In [210]:
full_tw_noH = full_tw[1:]
len(full_tw_noH)

1096

In [2]:
## copied the list form wikipedia
p_types = 'ISTJ,ISFJ,INFJ,INTJ,ISTP,ISFP,INFP,INTP,ESTP,ESFP,ENFP,ENTP,ESTJ,ESFJ,ENFJ,ENTJ'
## transformed into python list
p_list = p_types.split(',', 16)
#len(p_list)
p_list

['ISTJ',
 'ISFJ',
 'INFJ',
 'INTJ',
 'ISTP',
 'ISFP',
 'INFP',
 'INTP',
 'ESTP',
 'ESFP',
 'ENFP',
 'ENTP',
 'ESTJ',
 'ESFJ',
 'ENFJ',
 'ENTJ']

In [309]:
import sys
if sys.version_info[0] < 3:
    import got
else:
    import got3 as got


### Function to get users_screen_names

In [149]:
def get_users(phrase,num_tw):
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(phrase).setSince("2015-05-01").setUntil("2019-05-01").setMaxTweets(num_tw)
        tx = [got.manager.TweetManager.getTweets(tweetCriteria)[i].text for i in range(num_tw)]
        pl = [got.manager.TweetManager.getTweets(tweetCriteria)[i].permalink for i in range(num_tw)]
        return tx,pl

### test

In [151]:
get_users('I am INTJ',10)[1]

['https://twitter.com/thazin_98/status/1123340354978074625',
 'https://twitter.com/Catmask5/status/1123277859517112320',
 'https://twitter.com/rogue_odds1118/status/1123230822188953600',
 'https://twitter.com/kjellgal/status/1123229350504144903',
 'https://twitter.com/ohsixoneoh/status/1123172482565849090',
 'https://twitter.com/yaohngie/status/1123146583493660672',
 'https://twitter.com/modelerr/status/1123110028267835392',
 'https://twitter.com/artemidite/status/1122939904802136064',
 'https://twitter.com/TIL_ayg/status/1122924540584177664',
 'https://twitter.com/TIL_ayg/status/1122923788578869253']

In [152]:
for phr in query_search:
    tw_text = get_users(phr,10)[0]
    permalink = get_users(phr,10)[1]

In [153]:
permalink

['https://twitter.com/Roland00Address/status/1123323997272399872',
 'https://twitter.com/MBTIhorsewomen/status/1121636324111306752',
 'https://twitter.com/ashytin_/status/1121576457770131457',
 'https://twitter.com/shabda/status/1120630069234421761',
 'https://twitter.com/ziselianhua/status/1120093387435036672',
 'https://twitter.com/Willmur59880543/status/1119523566909702144',
 'https://twitter.com/AngieDicken/status/1119215504764350465',
 'https://twitter.com/TanisLeonhardi/status/1118963022096031745',
 'https://twitter.com/M_Y_Emerson/status/1118685860541685760',
 'https://twitter.com/clmurphy4/status/1116765611361587200']

In [154]:
tw_text

['Only selectively useful some of the time. It is either hyper-useful or not useful at all and it can get in the way of other things. Very polar. I am reminded of this. Some times they want to be an ENTJ , sometimes an INFJ, and other times an ESTP. https:// twitter.com/amor_vincit_1/ status/1122326654481600512 …',
 'I am having a sweet and dirty dream that a perfectly lovely gentle ginger ENTJ girl, throwing aside an aquamarine chemise, sings softly to an attentive but magical calico mustang.',
 'I, an humble ENTJ , today learned I am not good at fEeLiNgS',
 "I am an ENTJ . We don't believe in that shit.",
 'lmao the first ENTJ one I read just now is "I swear if I read one more description of ENTJ ’s being correlated to businesses, I’m going to lose my shit." a;dfjaslfjasf;ds "Help with workplace troubles!" " I am an ENTJ , and I want to know how to cope with severe stress."',
 "After discovering I am an entj personality type. It's like a ton of stuff finally makes sense. I just thoug

### Function to get users's tweets 

In [92]:
def get_user_tweets(name,num_tw):
        tweetCriteria = got.manager.TweetCriteria().setUsername(name).setMaxTweets(num_tw)
        return [got.manager.TweetManager.getTweets(tweetCriteria)[i].text for i in range(num_tw)]

#####
get_user_tweets('barackobama',5) 
#######
#create a list of user_ids
u_names = ['barackobama','Natallia112202']
user_id = []
i = 0

for item in u_names:
    try:
        user_id.append(api.get_user(screen_name = item).id)
    except:
        i = i + 1
        print(item)
        print("badvalue")
        user_id.append("NA")
print('Users IDs',user_id)

######
user_follow = []
i = 0

for item in u_names:
    try:
        user_follow.append(api.get_user(screen_name = item).followers_count)
    except:
        i = i + 1
        print(item)
        print("badvalue")
        user_id.append("NA")  
print("Users num of followers",user_follow)

In [141]:
### User's attributes choosen:
atr_list = ['id','location','description','protected','followers_count','friends_count',\
'favourites_count','time_zone','geo_enabled','profile_background_color',\
'profile_background_image_url','profile_image_url',\
'following']

### creating the attributes columns/lists

In [287]:
tweets = [get_user_tweets(name,4) for name in u_names]

In [288]:
tweets

[['The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn',
  'Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE',
  'Yes! Fourth star. Back to back. Congrats to the record breakers on the @USWNT , an incredible team that’s always pushing themselves—and the rest of us—to be even better. Love this team. #OneNationOneTeam',
  'Happy Fourth of July, everybody! This is always a great day in the Obama family: a chance to celebrate America—and Malia’s birthday, too. Hope all of you are able to get some time with friends, family, and fireworks. pic.twitter.com/Gn9kVCCnuf'],
 ['Wow. Leave ICE employees alone... They are clearly 

In [130]:

id_ = [api.get_user(screen_name = item).id for item in u_names]
loc = [api.get_user(screen_name = item).location for item in u_names]
descr = [api.get_user(screen_name = item).description for item in u_names]
prot = [api.get_user(screen_name = item).protected for item in u_names]
followers = [api.get_user(screen_name = item).followers_count for item in u_names]
friends = [api.get_user(screen_name = item).friends_count for item in u_names]             
fav = [api.get_user(screen_name = item).favourites_count for item in u_names]   
timeZ = [api.get_user(screen_name = item).time_zone for item in u_names] 
geo_en = [api.get_user(screen_name = item).geo_enabled for item in u_names] 
prof_bg_col = [api.get_user(screen_name = item).profile_background_color for item in u_names] 

prof_bkim_url = [api.get_user(screen_name = item).profile_background_image_url for item in u_names] 
prof_image_url = [api.get_user(screen_name = item).profile_image_url for item in u_names]
follow = [api.get_user(screen_name = item).following for item in u_names]



In [136]:
loc

['Washington, DC', '']

In [135]:
print(api.get_user('Natallia112202').profile_background_image_url)

None


In [125]:
followers

[107061925, 5]

In [137]:
col_list = ['id','location','description','protected','followers_count','friends_count',\
'favourites_count','time_zone','geo_enabled','profile_background_color',\
'profile_background_image_url','profile_image_url','default_profile_image',\
'following']




In [140]:
#### Extracting tweets 5 per user

In [88]:
u_names = ['barackobama','Natallia112202']
tweet_text = []
i = 0

for i in range(len(u_names)):
    
    tweet_text.append(get_user_tweets(u_names[i],5))
    
    print(i)
    print(tweet_text[i][0])

0
The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn
1
Wow. Leave ICE employees alone... They are clearly overwhelmed by illegals pouring through the border and doing their best. Unless you personally go and help them handling 1000+ diapers per day at the border, I recommend you just shut up.


In [90]:
tweet_text

[['The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn',
  'Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE',
  'Yes! Fourth star. Back to back. Congrats to the record breakers on the @USWNT , an incredible team that’s always pushing themselves—and the rest of us—to be even better. Love this team. #OneNationOneTeam',
  'Happy Fourth of July, everybody! This is always a great day in the Obama family: a chance to celebrate America—and Malia’s birthday, too. Hope all of you are able to get some time with friends, family, and fireworks. pic.twitter.com/Gn9kVCCnuf',
  'I’m admittedly biased, but this article gibes with m

In [63]:
tweetCriteria = got.manager.TweetCriteria().setUsername('barackobama').setMaxTweets(30)
obama_tw = []
for i in range(3):
    tweet1 = got.manager.TweetManager.getTweets(tweetCriteria)[i]
    obama_tw.append(tweet1.text)


    

In [72]:
new_Obama = [k.strip('\n') for k in obama_tw]

In [74]:
print(new_Obama)

['The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn', 'Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE', 'Yes! Fourth star. Back to back. Congrats to the record breakers on the @USWNT , an incredible team that’s always pushing themselves—and the rest of us—to be even better. Love this team. #OneNationOneTeam']


In [75]:
print(obama_tw)

['The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn', 'Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE', 'Yes! Fourth star. Back to back. Congrats to the record breakers on the @USWNT , an incredible team that’s always pushing themselves—and the rest of us—to be even better. Love this team. #OneNationOneTeam']


In [48]:
tw

['Proud to rep America’s best team! Congrats @USWNT and thanks for being such a strong inspiration for women and girls—and everybody—all across the country. pic.twitter.com/9Gp6UTZhEE',
 '#personalityassessment https:// lnkd.in/g6Amrp7']

In [31]:
tweet1.text

'The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn'

In [26]:
import sys
if sys.version_info[0] < 3:
    import got
else:
    import got3 as got

def main():

    def printTweet(descr, t):
        print(descr)
        print("Username: %s" % t.username)
        print("Retweets: %d" % t.retweets)
        print("Text: %s" % t.text)
        print("Mentions: %s" % t.mentions)
        print("Hashtags: %s\n" % t.hashtags)

# Example 1 - Get tweets by username
    tweetCriteria = got.manager.TweetCriteria().setUsername('barackobama').setMaxTweets(1)
    tweet1 = got.manager.TweetManager.getTweets(tweetCriteria)[0]
    return tweet1

    #printTweet("### Example 1 - Get tweets by username [barackobama]", tweet)
    
    # Example 2 - Get tweets by query search
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('europe refugees').setSince("2015-05-01").setUntil("2015-09-30").setMaxTweets(1)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]

    printTweet("### Example 2 - Get tweets by query search [europe refugees]", tweet)

    # Example 3 - Get tweets by username and bound dates
    tweetCriteria = got.manager.TweetCriteria().setUsername("barackobama").setSince("2015-09-10").setUntil("2015-09-12").setMaxTweets(1)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]

    printTweet("### Example 3 - Get tweets by username and bound dates [barackobama, '2015-09-10', '2015-09-12']", tweet)

if __name__ == '__main__':
    r = main()


In [28]:
r.text

'The @ObamaFoundation is helping a new generation of leaders make their mark, and there’s an extraordinary group gathering this week for Obama Leaders: Africa. Their work will help unlock the incredible promise of the continent—and make the world better for all of us. pic.twitter.com/rnPHn9UuEn'